# Final Model

## Setup dello Script e Caricamento dei Modelli

- Caricamento del modello di object recognition.
- Caricamento del face detection con Haar Cascades.
- Caricamento dei modelli per text recognition e number recognition.
- Caricamento del chatbot per le domande sull’immagine.

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.activations import softmax
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import torch.nn.functional as F
import torch
import torch.nn as nn
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disabilita la GPU

2024-11-05 01:44:44.008595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730767484.032560   11424 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730767484.040328   11424 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 01:44:44.063919: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Percorsi dei modelli
CHATBOT_MODEL_PATH = "/home/pierluigi/Documents/models/Chatbot/vqa_model.h5"
FACE_CASCADE_PATH = "/home/pierluigi/Documents/models/Face Detenction/haarcascades/haarcascade_frontalface_default.xml"
NUMBER_RECOGNITION_MODEL_PATH = "/home/pierluigi/Documents/models/Number Recognition/number_recognition_model.h5"
TEXT_RECOGNITION_MODEL_PATH = "/home/pierluigi/Documents/models/Text Recognition/text_recognition_model.h5"
OBJECT_RECOGNITION_MODEL_PATH = "/home/pierluigi/Documents/models/Object Recognition/object_recognition_model.pth"

In [3]:
# Caricamento del face detector
face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)
print("Modello Face Detection caricato correttamente.")

Modello Face Detection caricato correttamente.


In [4]:
# Caricamento del modello di riconoscimento dei numeri con custom_objects
number_recognition_model = load_model(
    NUMBER_RECOGNITION_MODEL_PATH,
    custom_objects={'softmax_v2': softmax}
)

print("Modello Number Recognition caricato correttamente.")

2024-11-05 01:44:48.446690: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-05 01:44:48.446730: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: luirio-penguin
2024-11-05 01:44:48.446737: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: luirio-penguin
2024-11-05 01:44:48.446827: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 560.35.3
2024-11-05 01:44:48.446843: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 560.35.3
2024-11-05 01:44:48.446847: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 560.35.3


Modello Number Recognition caricato correttamente.


In [5]:
# Caricamento del modello Chatbot
chatbot_model = load_model(CHATBOT_MODEL_PATH)
print("Modello Chatbot caricato correttamente.")

Modello Chatbot caricato correttamente.


In [14]:
import torch
import torch.nn as nn

class ObjectRecognitionModel(nn.Module):
    def __init__(self):
        super(ObjectRecognitionModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # layers.0
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # layers.2
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # layers.4
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),  # layers.6
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),  # layers.8 (modificato per avere output di 128 canali)
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # layers.10 (input 128, output 256)
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 16 * 16, 1024),  # layers.13
            nn.ReLU(),
            nn.Linear(1024, 512),            # layers.15
            nn.ReLU(),
            nn.Linear(512, 10)               # layers.17
        )

    def forward(self, x):
        return self.layers(x)

# Istanzia il modello
object_recognition_model = ObjectRecognitionModel()

# Carica il `state_dict`, rimuovendo il prefisso "network." e sostituendolo con "layers."
state_dict = torch.load("/home/pierluigi/Documents/models/Object Recognition/object_recognition_model.pth", map_location=torch.device('cpu'))
new_state_dict = {k.replace("network.", "layers."): v for k, v in state_dict.items()}  # Rimuove "network." e lo sostituisce con "layers."

# Carica i pesi nel modello
object_recognition_model.load_state_dict(new_state_dict, strict=False)
object_recognition_model.eval()  # Imposta il modello in modalità di valutazione

print("Modello Object Recognition caricato correttamente.")


Modello Object Recognition caricato correttamente.


/tmp/ipykernel_11424/58763373.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/pierluigi/Documents/models/Object Recognition/object_recogn